<a href="https://colab.research.google.com/github/Shaheer-Khan/AISem3/blob/master/CW/Home_Credit_default_risk_CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shaheersk","key":"7d251b05c20db9fd4c81f313f4770bcd"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!pip uninstall -y kaggle
!pip install —upgrade pip
!pip install kaggle==1.5.6
!kaggle -v

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
ERROR: Invalid requirement: '—upgrade'
     |████████████████████████████████| 61kB 2.1MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=fb5566a460a3a7846fb11a4b701d5b8bc64f5fc583737d519266dc34485a582a
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
Kaggle API 1.5.6


In [6]:
!kaggle competitions download -c home-credit-default-risk

 99% 681M/688M [00:07<00:00, 97.5MB/s]
100% 688M/688M [00:07<00:00, 95.4MB/s]


In [7]:
from zipfile import ZipFile

file_name = "home-credit-default-risk.zip"

with ZipFile(file_name,"r") as zip:
  zip.extractall()
  print('done')

done


In [8]:
import numpy as np 
import pandas as pd

In [11]:
train = pd.read_csv('application_train.csv')
bureau = pd.read_csv('bureau.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')
credit_card_balance = pd.read_csv('credit_card_balance.csv')
installments_payments = pd.read_csv('installments_payments.csv')
previous_application = pd.read_csv('previous_application.csv')
POS_CASH_balance = pd.read_csv('POS_CASH_balance.csv')

test = pd.read_csv('application_test.csv')
sample_sub = pd.read_csv('sample_submission.csv')

In [17]:
train.head(10)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,...,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,...,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,-2437,NaN,1,1,0,1,0,0,Laborers,2.0,2,2,WEDNESDAY,17,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,-3458,NaN,1,1,0,1,0,0,Core staff,1.0,2,2,THURSDAY,11,0,0,0,0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-1106.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
5,100008,0,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,454500.0,"Spouse, partner",State servant,Secondary / secondary special,Married,House / apartment,0.035792,-16941,-1588,-4970.0,-477,NaN,1,1,1,1,1,0,Laborers,2.0,2,2,WEDNESDAY,16,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-2536.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0
6,100009,0,Cash loans,F,Y,Y,1,171000.0,1560726.0,41301.0,1395000.0,Unaccompanied,Commercial associate,Higher education,Married,House / apartment,0.035792,-13778,-3130,-1213.0,-619,17.0,1,1,0,1,1,0,Acco

In [19]:
x = [train,test]

In [24]:
Train_test =  pd.concat(x)

Train_test.sort_values(by = 'SK_ID_CURR' , ascending = True, inplace = True)

In [26]:
print(train.shape)
print(test.shape)
print(Train_test.shape)

(307511, 122)
(48744, 121)
(356255, 122)


In [25]:
Train_test.head(10)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,NaN,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,-19241,-2329,-5170.0,-812,NaN,1,1,0,1,0,1,NaN,2.0,2,2,TUESDAY,18,0,0,0,0,0,0,...,NaN,0.0514,NaN,NaN,NaN,block of flats,0.0392,"Stone, brick",No,0.0,0.0,0.0,0.0,-1740.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,...,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,...,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,NaN,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-18064,-4469,-9118.0,-1623,NaN,1,1,0,1,0,0,Low-skill Laborers,2.0,2,2,FRIDAY,9,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,-2437,NaN,1,1,0,1,0,0,Laborers,2.0,2,2,WEDNESDAY,17,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,-3458,NaN,1,1,0,

In [ ]:
bureau_merge = pd.merge(left=bureau,
                      right=bureau_balance,
                      how="left",
                      left_on="SK_ID_BUREAU",
                      right_index=True)

In [ ]:
print(bureau.shape)
print(bureau_balance.shape)
print(bureau_merge.shape)

(1716428, 17)
(27299925, 3)
(1716428, 20)


In [ ]:
bureau_merge.duplicated(['SK_ID_CURR']).sum()

In [ ]:
bureau_merge.head(10)

,SK_ID_CURR,SK_ID_BUREAU_x,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,SK_ID_BUREAU_y,MONTHS_BALANCE,STATUS
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.00,NaN,0.0,Consumer credit,-131,NaN,6066882,-24,0
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.00,NaN,0.0,Credit card,-20,NaN,6066882,-25,0
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN,6066882,-26,0
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN,6066884,0,C
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN,6066884,-1,C
5,215354,5714467,Active,currency 1,-273,0,27460.0,NaN,0.0,0,180000.00,71017.38,108982.62,0.0,Credit card,-31,NaN,6066884,-2,C
6,215354,5714468,Active,currency 1,-43,0,79.0,NaN,0.0,0,42103.80,42103.80,0.00,0.0,Consumer credit,-22,NaN,6066884,-3,C
7,162297,5714469,Closed,currency 1,-1896,0,-1684.0,-1710.0,14985.0,0,76878.45,0.00,0.00,0.0,Consumer credit,-1710,NaN,6066884,-4,C
8,162297,5714470,Closed,currency 1,-1146,0,-811.0,-840.0,0.0,0,103007.70,0.00,0.00,0.0,Consumer credit,-840,NaN,6066884,-5,C
9,162297,5714471,Active,currency 1,-1146,0,-484.0,NaN,0.0,0,4500.00,0.00,0.00,0.0,Credit card,-690,NaN,6066884,-6,C


In [ ]:
bureau_credit_merge = pd.merge(left=credit_card_balance,
                      right=bureau_merge,
                      how="left",
                      left_on="SK_ID_CURR",
                      right_index=True)

In [ ]:
print(bureau_merge.shape)
print(credit_card_balance.shape)
print(bureau_credit_merge.shape)

(1716428, 20)
(3840312, 23)
(3840312, 43)


In [ ]:
bureau_credit_merge.head(10)

,SK_ID_CURR_x,SK_ID_BUREAU_x,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,SK_ID_BUREAU_y,MONTHS_BALANCE_x,STATUS,SK_ID_PREV,SK_ID_CURR_y,MONTHS_BALANCE_y,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.00,NaN,0.0,Consumer credit,-131,NaN,6066882,-24,0,2429017,207864,-76,95380.83,90000,0.0,0.0,0.0,0.0,5400.0,0.0,0.0,87527.88,95380.83,95380.83,0.0,0,0.0,0.0,38.0,Active,1,1
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.00,NaN,0.0,Credit card,-20,NaN,6066882,-25,0,2429017,207864,-76,95380.83,90000,0.0,0.0,0.0,0.0,5400.0,0.0,0.0,87527.88,95380.83,95380.83,0.0,0,0.0,0.0,38.0,Active,1,1
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN,6066882,-26,0,2429017,207864,-76,95380.83,90000,0.0,0.0,0.0,0.0,5400.0,0.0,0.0,87527.88,95380.83,95380.83,0.0,0,0.0,0.0,38.0,Active,1,1
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN,6066884,0,C,2429017,207864,-76,95380.83,90000,0.0,0.0,0.0,0.0,5400.0,0.0,0.0,87527.88,95380.83,95380.83,0.0,0,0.0,0.0,38.0,Active,1,1
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN,6066884,-1,C,2429017,207864,-76,95380.83,90000,0.0,0.0,0.0,0.0,5400.0,0.0,0.0,87527.88,95380.83,95380.83,0.0,0,0.0,0.0,38.0,Active,1,1
5,215354,5714467,Active,currency 1,-273,0,27460.0,NaN,0.0,0,180000.00,71017.38,108982.62,0.0,Credit card,-31,NaN,6066884,-2,C,2429017,207864,-76,95380.83,90000,0.0,0.0,0.0,0.0,5400.0,0.0,0.0,87527.88,95380.83,95380.83,0.0,0,0.0,0.0,38.0,Active,1,1
6,215354,5714468,Active,currency 1,-43,0,79.0,NaN,0.0,0,42103.80,42103.80,0.00,0.0,Consumer credit,-22,NaN,6066884,-3,C,2429017,207864,-76,95380.83,90000,0.0,0.0,0.0,0.0,5400.0,0.0,0.0,87527.88,95380.83,95380.83,0.0,0,0.0,0.0,38.0,Active,1,1
7,162297,5714469,Closed,currency 1,-1896,0,-1684.0,-1710.0,14985.0,0,76878.45,0.00,0.00,0.0,Consumer credit,-1710,NaN,6066884,-4,C,2490759,179704,-9,0.00,45000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.00,0.00,0.00,NaN,0,NaN,NaN,0.0,Active,0,0
8,162297,5714470,Closed,currency 1,-1146,0,-811.0,-840.0,0.0,0,103007.70,0.00,0.00,0.0,Consumer credit,-840,NaN,6066884,-5,C,2490759,179704,-9,0.00,45000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.00,0.00,0.00,NaN,0,NaN,NaN,0.0,Active,0,0
9,162297,5714471,Active,currency 1,-1146,0,-484.0,NaN,0.0,0,4500.00,0.00,0.00,0.0,Credit card,-690,NaN,6066884,-6,C,2490759,179704,-9,0.00,45000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.00,0.00,0.00,NaN,0,NaN,NaN,0.0,Active,0,0


In [ ]:
bureau_c_installment_merge = pd.merge(left=installments_payments,
                      right=bureau_credit_merge,
                      how="left",
                      left_on="SK_ID_CURR",
                      right_index=True)

In [ ]:
print(installments_payments.shape)
print(bureau_credit_merge.shape)
print(bureau_c_installment_merge.shape)

(13605401, 8)
(3840312, 43)
(13605401, 51)


In [ ]:
bureau_c_installment_merge.head(10)

,SK_ID_PREV_x,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,SK_ID_PREV_y,SK_ID_CURR_x,MONTHS_BALANCE_x,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,SK_ID_CURR_y,SK_ID_BUREAU_x,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,SK_ID_BUREAU_y,MONTHS_BALANCE_y,STATUS
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360,1462197,377916,-22,34554.510,180000,0.0,0.0,0.0,0.0,9000.000,9000.000,9000.000,31003.290,34554.510,34554.510,0.0,0,0.0,0.0,74.0,Active,0,0,258433,5994626,Closed,currency 1,-2253,0,-2029.0,-2017.0,NaN,0,102928.5,0.00,0.00,0.0,Consumer credit,-955,NaN,6447421,-52,0
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525,1932364,207042,-19,0.000,765000,NaN,0.0,NaN,NaN,0.000,NaN,0.000,0.000,0.000,0.000,NaN,0,NaN,NaN,0.0,Signed,0,0,236313,6573958,Closed,currency 1,-1647,0,-1525.0,-1517.0,NaN,0,19287.0,0.00,0.00,0.0,Consumer credit,-1024,0.00,6134136,-8,C
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000,1213596,142757,-17,0.000,337500,0.0,0.0,0.0,0.0,655.965,2473.155,2342.655,0.000,0.000,0.000,0.0,0,0.0,0.0,8.0,Active,0,0,451704,6560313,Closed,currency 1,-800,0,-619.0,-613.0,NaN,0,133699.5,0.00,0.00,0.0,Consumer credit,-612,34108.38,5743199,-42,X
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130,1201945,394332,-76,270330.930,270000,0.0,0.0,0.0,0.0,13500.000,13500.000,13500.000,262002.060,270330.930,270330.930,0.0,0,0.0,0.0,24.0,Active,0,0,117557,6322002,Closed,currency 1,-151,0,NaN,-131.0,NaN,0,6714.0,NaN,NaN,0.0,Microloan,-113,0.00,6147241,-16,C
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585,1531488,260732,-19,0.000,225000,NaN,0.0,NaN,NaN,0.000,NaN,0.000,0.000,0.000,0.000,NaN,0,NaN,NaN,0.0,Active,0,0,145822,5891734,Closed,currency 1,-2252,0,-2161.0,-2132.0,NaN,0,18432.0,0.00,0.00,0.0,Consumer credit,-917,0.00,6527295,-28,0
5,1137312,164489,1.0,12,-1384.0,-1417.0,5970.375,5970.375,2701386,316287,-74,102330.540,135000,0.0,0.0,0.0,0.0,6750.000,6750.000,6750.000,98751.645,102330.540,102330.540,0.0,0,0.0,0.0,14.0,Active,0,0,198287,6066318,Closed,currency 1,-635,0,10312.0,-178.0,3780.0,0,90000.0,NaN,NaN,0.0,Credit card,-178,NaN,6399638,-5,C
6,2234264,184693,4.0,11,-349.0,-352.0,29432.295,29432.295,2691264,363433,-45,138160.080,135000,0.0,0.0,0.0,0.0,6750.000,6975.000,6975.000,133906.140,138160.080,138160.080,0.0,0,0.0,0.0,51.0,Active,1,1,306500,6829517,Active,currency 1,-132,0,27374.0,NaN,0.0,0,22500.0,18003.33,4496.67,0.0,Credit card,-27,4500.00,5680202,-61,X
7,1818599,111420,2.0,4,-968.0,-994.0,17862.165,17862.165,1496934,395009,-90,179236.035,180000,22500.0,115200.0,92700.0,0.0,3375.000,3375.000,3375.000,174827.610,179236.035,179236.035,1.0,2,1.0,0.0,4.0,Active,0,0,173441,6322826,Active,currency 1,-293,0,11.0,NaN,0.0,0,137115.0,15601.50,0.00,0.0,Consumer credit,-18,NaN,6069191,-56,1
8,2723183,112102,0.0,14,-197.0,-197.0,70.740,70.740,1041897,206896,-57,70414.155,90000,9000.0,9000.0,0.0,0.0,4500.000,13500.000,13500.000,66285.675,70414.155,70414.155,1.0,1,0.0,0.0,39.0,Active,0,0,275761,6573791,Closed,currency 1,-113,0,-97.0,-104.0,NaN,0,13500.0,0.00,0.00,0.0,Consumer credit,-98,0.00,6134132,-52,X
9,1413990,109741,1.0,4,-570.0,-609.0,14308.470,14308.470,1981186,232822,-3,0.000,45000,0.0,0.0,0.0,0.0,0.000,292.950,0.000,0.000,0.000,0.000,0.0,0,0.0,0.0,7.0,Active,0,0,285240,5848108,Active,currency 1,-1491,0,335.0,NaN,NaN,0,243000.0,66267.00,0.0

In [ ]:
bureau_c_in_previous_merge = pd.merge(left=previous_application,
                      right=bureau_c_installment_merge,
                      how="left",
                      left_on="SK_ID_CURR",
                      right_index=True)

In [ ]:
print(previous_application.shape)
print(bureau_c_installment_merge.shape)
print(bureau_c_in_previous_merge.shape)

NameError: ignored